In [0]:
import sys
import os
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob


import spacy
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
% matplotlib inline
import json

from datetime import datetime, timedelta
from IPython.display import clear_output
from tqdm import tqdm


from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS

In [0]:
#@title Enter Twitter Credentials
TWITTER_KEY = '' #@param {type:"string"}
TWITTER_SECRET_KEY = '' #@param {type:"string"}

In [0]:
# Authenticate
auth = tweepy.AppAuthHandler(TWITTER_KEY, TWITTER_SECRET_KEY)

api = tweepy.API(auth, wait_on_rate_limit=True,
				   wait_on_rate_limit_notify=True)

if (not api):
    print ("Can't Authenticate")
    sys.exit(-1)


In [141]:
#@markdown ###Get All tweets

searchQuery = "PSG "
tweetsPerQry = 100  # this is the max the API permits

maxTweets = 10000
new_tweets = api.search(q=searchQuery, count=tweetsPerQry, lang="fr")
topic = searchQuery[0:searchQuery.find('-')].capitalize().strip()
print(topic)

Psg


In [142]:
#@markdown ### Twitter json format
print(json.dumps(new_tweets[3]._json,indent=8))


{
        "created_at": "Wed Feb 26 21:42:14 +0000 2020",
        "id": 1232782953429819392,
        "id_str": "1232782953429819392",
        "text": "Eh merde !!!  On se fait manger en 5 min alors qu'on m\u00e8ne, comme face au PSG... \ud83e\udd26\ud83c\udffd\u200d\u2642",
        "truncated": false,
        "entities": {
                "hashtags": [],
                "symbols": [],
                "user_mentions": [],
                "urls": []
        },
        "metadata": {
                "iso_language_code": "fr",
                "result_type": "recent"
        },
        "source": "<a href=\"http://twitter.com/download/android\" rel=\"nofollow\">Twitter for Android</a>",
        "in_reply_to_status_id": null,
        "in_reply_to_status_id_str": null,
        "in_reply_to_user_id": null,
        "in_reply_to_user_id_str": null,
        "in_reply_to_screen_name": null,
        "user": {
                "id": 3035472825,
                "id_str": "3035472825",
                "n

In [143]:
tweet_list = []
for tweet in range(len(new_tweets)):
  retweet_text = None
  if "RT" in new_tweets[tweet]._json['text']:
    if 'retweeted_status' in new_tweets[tweet]._json.keys():
      retweet_text = new_tweets[tweet]._json['retweeted_status']['text']

  topic = searchQuery[0:searchQuery.find('-')].capitalize().strip()
  tweet_user = new_tweets[tweet]._json['user']['name']
  isRetweet = new_tweets[tweet]._json["retweeted"]
  tweet_list.append([tweet_user, topic, isRetweet, retweet_text])

pd.DataFrame(tweet_list,columns=["user", "topic", "isRetweet", "retweeted"])
      

,user,topic,isRetweet,retweeted
0,El Matador❤️🥺,Psg,False,None
1,24 . 14 . 2 💜,Psg,False,Vous croyez vraiment que c’est le seul y’a des...
2,Tuchel Out,Psg,False,None
3,𝖓𝖔𝖘𝖒𝖆𝕾🇨🇮,Psg,False,None
4,ALLEZ PARIS 🇫🇷⭐⭐🏆,Psg,False,None
...,...,...,...,...
95,Seugn Teunkou ⚔️,Psg,False,MATRIX 😎 https://t.co/7lIMQiQ1Uw
96,Dylan,Psg,False,Kalidou Koulibaly se verrait bien succéder à T...
97,2launax,Psg,False,L’impression de voir le PSG en regardant la Ju...
98,bilelgouss95,Psg,False,Angel Di Maria a annoncé au Magazine du PSG vo...
